In [1]:
import pandas as pd
# import numpy as np
import os
import sys

dir_path = '/home/vasant/projects/Pride-of-Sahyadri'
sys.path.append("/home/vasant/projects/Pride-of-Sahyadri")
os.chdir(dir_path)

In [ ]:
df = pd.read_csv('data/maharashtra-forts.csv')
df.sample(3)

In [ ]:
df.info()

## Option 1 — Import Routers & Call Functions Directly

In [12]:
from src.core.data_loader import load_forts
from src.api.routers.forts import list_forts, get_fort

In [13]:
# Test data loading
df = load_forts()
df.head()

,fort_id,name,alternate_names,district,taluka,latitude,longitude,base_village,type,elevation_m,...,built_by,year_of_construction,key_events,trek_difficulty,trek_time_hours,best_season,water_availability,accommodation,asi_protected,notes
0,1,Aad Fort,,Ratnagiri,Dapoli,17.7125,73.2391,Aad,Giri Durg (Hill Fort),250,...,Unknown,Unknown,Minor watchtower fort,Easy,1.0,Winter,Information Not Available,Information Not Available,FALSE,A small lesser-known fort covered in dense ve...
1,2,Achala Fort,,Nashik,Satana,20.4578,74.0536,Pimpalgaon,Giri Durg (Hill Fort),1372,...,Shivaji Maharaj,c. 17th Century,Part of Satmala hill range forts; Captured by ...,Medium,2.0,Winter,Seasonal,Information Not Available,FALSE,"Twin fort to Ahivantgad, offers great views of..."
2,3,Achalpur Fort,Ellichpur Fort,Amravati,Achalpur,21.2612,77.5143,Achalpur,Bhuikot (Land Fort),358,...,Ahmad Shah Bahmani,c. 1425,Capital of Berar Sultanate,Easy,0.5,Winter,Information Not Available,Information Not Available,TRUE,"Located inside the city, known for its massive..."
3,4,Adas Fort,,Satara,Khatav,17.6534,74.6543,Adas,Giri Durg (Hill Fort),937,...,Unknown,c. 17th Century,Minor watchtower fort,Easy,1.0,Post-Monsoon,Information Not Available,Information Not Available,FALSE,A small fort primarily used as a watch post.
4,5,Ahiwantgad,,Nashik,Satana,20.4533,74.0494,Pimpalgaon,Giri Durg (Hill Fort),1385,...,Shivaji Maharaj,c. 17th Century,Treaty of Purandar; Captured by British,Medium,2.0,Winter,Year-round in cisterns,Caves,FALSE,"Twin fort to Achala fort, features large caves..."


In [ ]:
result = list_forts(limit=5, q=None, district=None)
result

In [ ]:
result[0]

In [ ]:
result = list_forts(limit=10, q="shivaji", district=None)
result

In [ ]:
from src.api.routers.clustering import get_clusters, predict_cluster

get_clusters()

In [ ]:
predict_cluster(lat=18.52, lon=73.85)

In [ ]:
from src.core.rag_engine import RAGEngine

rag = RAGEngine()
rag.load_data(df=df)
rag.build_index()

In [ ]:
result = rag.query("sea fort with strong historical importance", k=3)
result

In [11]:
from src.core.recommender import recommend_by_proximity, recommend_similar

In [ ]:
df0 = recommend_by_proximity(df, lat=18.52, lon=73.85, k=10)
df0

In [ ]:
df2 = recommend_similar(df, fort_id=1, k=5)
df2.head(5)

## Option 2 — Use FastAPI TestClient

In [ ]:
from fastapi.testclient import TestClient
from src.api.main import app

client = TestClient(app=app)

In [ ]:
r = client.get("/forts")
print(r.status_code)
r.json()[:3]

In [ ]:
client.get("/forts", params={"q": "Sindhudurg"}).json()[:3]

In [ ]:
client.get(
    "/search/semantic_search",
    params={"q": "sea forts built by shivaji"}
).json()

In [ ]:
tests = {
    "forts_list": client.get("/forts"),
    "search": client.get("/forts", params={"q":"hill"}),
    "single_fort": client.get("/forts/1"),
    "cluster_summary": client.get("/clusters"),
    "cluster_predict": client.get("/clusters/predict", params={"lat":18.5,"lon":73.8}),
    "nearby": client.get("/recommend/nearby", params={"lat":18.5,"lon":73.8,"k":3}),
    "similar": client.get("/recommend/similar/10", params={"k":3}),
    "rag_search": client.get("/search/semantic_search", params={"q":"sea fort in Sindhudurg district"})
}

for name, result in tests.items():
    print(f"\n--- {name.upper()} ---")
    print("Status:", result.status_code)
    print("Sample:", result.json()[:2] if isinstance(result.json(), list) else result.json())


In [ ]:
import requests
import pandas as pd

BASE = "http://localhost:8000"

def show(x):
    """Pretty print JSON or DataFrame intelligently."""
    if isinstance(x, list):
        if len(x) == 0:
            print("[] (empty list)")
        else:
            display(pd.DataFrame(x).head())
    elif isinstance(x, dict):
        print(x)
    else:
        print(x)


In [ ]:
try:
    r = requests.get(f"{BASE}/forts")
    print("STATUS:", r.status_code)
except Exception as e:
    print("❌ Cannot reach backend:", e)

In [ ]:
forts = requests.get(f"{BASE}/forts").json()
show(forts)

In [ ]:
clusters = requests.get(f"{BASE}/data")
print("Cluster Counts:")
print(clusters)

In [2]:
from fastapi.testclient import TestClient
from src.api.main import app

client = TestClient(app=app)

/home/vasant/projects/Pride-of-Sahyadri/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RAGEngine: Corpus created with 346 entries.
RAGEngine: Loading cached embeddings...
RAGEngine: Embeddings loaded from cache.


In [3]:
client.get("/clusters").json()

{'0': 89, '1': 67, '2': 18, '3': 70, '4': 79, '5': 23}

In [4]:
clustered = client.get("/clusters/data").json()

In [22]:
clustered[0]

{'fort_id': 1,
 'name': 'Aad Fort',
 'alternate_names': 'Information Not Available',
 'district': 'Ratnagiri',
 'taluka': 'Dapoli',
 'latitude': 17.7125,
 'longitude': 73.2391,
 'base_village': 'Aad',
 'type': 'Giri Durg (Hill Fort)',
 'elevation_m': 250,
 'current_condition': 'In Ruins',
 'era': 'Maratha Empire',
 'built_by': 'Unknown',
 'year_of_construction': 'Unknown',
 'key_events': 'Minor watchtower fort',
 'trek_difficulty': 'Easy',
 'trek_time_hours': 1.0,
 'best_season': 'Winter',
 'water_availability': 'Information Not Available',
 'accommodation': 'Information Not Available',
 'asi_protected': 'FALSE',
 'notes': 'A small  lesser-known fort covered in dense vegetation.',
 'difficulty_num': 1.0,
 'cluster': 0}

In [6]:
client.get("/recommend/nearby", params={"lat":18.5,"lon":73.8,"k":3}).json()


[{'fort_id': 249,
  'name': 'Pisol Fort',
  'alternate_names': '',
  'district': 'Pune',
  'taluka': 'Haveli',
  'latitude': 18.4231,
  'longitude': 73.8312,
  'base_village': 'Pisol',
  'type': 'Giri Durg (Hill Fort)',
  'elevation_m': 850,
  'current_condition': 'Completely Ruined',
  'era': 'Maratha Empire',
  'built_by': 'Unknown',
  'year_of_construction': 'Unknown',
  'key_events': 'A very small watchtower',
  'trek_difficulty': 'Easy',
  'trek_time_hours': 0.5,
  'best_season': 'Winter',
  'water_availability': 'Information Not Available',
  'accommodation': 'Information Not Available',
  'asi_protected': 'FALSE',
  'notes': 'Only a small plinth and a water cistern remain.',
  'distance_km': 9.127436840115315},
 {'fort_id': 101,
  'name': 'Ghotavade Fort',
  'alternate_names': '',
  'district': 'Pune',
  'taluka': 'Mulshi',
  'latitude': 18.5412,
  'longitude': 73.6815,
  'base_village': 'Ghotavade',
  'type': 'Giri Durg (Hill Fort)',
  'elevation_m': 700,
  'current_condition':

In [9]:
client.get("/recommend/similar/2", params={"k":3}).json()

[{'fort_id': 2,
  'name': 'Achala Fort',
  'alternate_names': '',
  'district': 'Nashik',
  'taluka': 'Satana',
  'latitude': 20.4578,
  'longitude': 74.0536,
  'base_village': 'Pimpalgaon',
  'type': 'Giri Durg (Hill Fort)',
  'elevation_m': 1372,
  'current_condition': 'In Ruins',
  'era': 'Maratha Empire',
  'built_by': 'Shivaji Maharaj',
  'year_of_construction': 'c. 17th Century',
  'key_events': 'Part of Satmala hill range forts; Captured by British in 1818',
  'trek_difficulty': 'Medium',
  'trek_time_hours': 2.0,
  'best_season': 'Winter',
  'water_availability': 'Seasonal',
  'accommodation': 'Information Not Available',
  'asi_protected': 'FALSE',
  'notes': 'Twin fort to Ahivantgad, offers great views of the region.',
  'type_score': 1,
  'elev_diff': 0,
  'score': 1.0},
 {'fort_id': 137,
  'name': 'Kamalgad',
  'alternate_names': '',
  'district': 'Satara',
  'taluka': 'Wai',
  'latitude': 17.895,
  'longitude': 73.785,
  'base_village': 'Tupewadi',
  'type': 'Giri Durg (Hi

In [ ]:
clustered[25]['fort_id']

{'fort_id': 26,
 'name': 'Bahadurgad',
 'alternate_names': 'Information Not Available',
 'district': 'Ahilyanagar',
 'taluka': 'Shrigonda',
 'latitude': 18.7831,
 'longitude': 74.6508,
 'base_village': 'Pedgaon',
 'type': 'Bhuikot (Land Fort)',
 'elevation_m': 540,
 'current_condition': 'In Ruins',
 'era': 'Mughal Empire',
 'built_by': 'Aurangzeb',
 'year_of_construction': 'c. 1672',
 'key_events': "Aurangzeb's major Deccan campaign base; renamed Bahadurgad from Pedgaon",
 'trek_difficulty': 'Easy',
 'trek_time_hours': 0.5,
 'best_season': 'Winter',
 'water_availability': 'Information Not Available',
 'accommodation': 'Information Not Available',
 'asi_protected': 'FALSE',
 'notes': 'Known for its Bhuleshwar temple and historical significance during Mughal-Maratha wars.',
 'difficulty_num': 1.0,
 'cluster': 4}

In [18]:
fort_id = clustered[25]['fort_id']  # or if fort_id exists: clustered[0]["fort_id"]

similar = client.get(f"/recommend/similar/{fort_id}", params={"k": 3}).json()

similar

[{'fort_id': 26,
  'name': 'Bahadurgad',
  'alternate_names': '',
  'district': 'Ahilyanagar',
  'taluka': 'Shrigonda',
  'latitude': 18.7831,
  'longitude': 74.6508,
  'base_village': 'Pedgaon',
  'type': 'Bhuikot (Land Fort)',
  'elevation_m': 540,
  'current_condition': 'In Ruins',
  'era': 'Mughal Empire',
  'built_by': 'Aurangzeb',
  'year_of_construction': 'c. 1672',
  'key_events': "Aurangzeb's major Deccan campaign base; renamed Bahadurgad from Pedgaon",
  'trek_difficulty': 'Easy',
  'trek_time_hours': 0.5,
  'best_season': 'Winter',
  'water_availability': 'Information Not Available',
  'accommodation': 'Information Not Available',
  'asi_protected': 'FALSE',
  'notes': 'Known for its Bhuleshwar temple and historical significance during Mughal-Maratha wars.',
  'type_score': 1,
  'elev_diff': 0,
  'score': 1.0},
 {'fort_id': 246,
  'name': 'Piliv Fort',
  'alternate_names': '',
  'district': 'Solapur',
  'taluka': 'Malshiras',
  'latitude': 17.9712,
  'longitude': 74.9513,
  

In [20]:
client.get("/search/semantic_search", params={"q":"Which forts are difficult treks?"}).json()

[{'fort_id': 12,
  'name': 'Alang Fort',
  'alternate_names': '',
  'district': 'Nashik',
  'taluka': 'Igatpuri',
  'latitude': 19.5855,
  'longitude': 73.6651,
  'base_village': 'Kulangwadi',
  'type': 'Giri Durg (Hill Fort)',
  'elevation_m': 1479,
  'current_condition': 'In Ruins',
  'era': 'Maratha Empire',
  'built_by': 'Unknown',
  'year_of_construction': 'c. 17th-18th Century',
  'key_events': 'Part of the intense Alang-Madan-Kulang trek',
  'trek_difficulty': 'Hard',
  'trek_time_hours': 5.0,
  'best_season': 'Winter',
  'water_availability': 'Seasonal in cisterns',
  'accommodation': 'Caves',
  'asi_protected': 'FALSE',
  'notes': 'Part of the toughest trek in the Sahyadris; requires technical climbing gear.'},
 {'fort_id': 58,
  'name': 'Birwadi Fort',
  'alternate_names': '',
  'district': 'Raigad',
  'taluka': 'Mahad',
  'latitude': 18.2227,
  'longitude': 73.3985,
  'base_village': 'Birwadi',
  'type': 'Giri Durg (Hill Fort)',
  'elevation_m': 160,
  'current_condition': '

In [23]:
from src.core.cluster_engine import ClusterEngine

cluster_engine = ClusterEngine()
cluster_engine.build_clusters()

(     fort_id                 name            alternate_names     district  \
 0          1             Aad Fort  Information Not Available    Ratnagiri   
 1          2          Achala Fort  Information Not Available       Nashik   
 2          3        Achalpur Fort             Ellichpur Fort     Amravati   
 3          4            Adas Fort  Information Not Available       Satara   
 4          5           Ahiwantgad  Information Not Available       Nashik   
 ..       ...                  ...                        ...          ...   
 341      342         Visapur Fort  Information Not Available         Pune   
 342      343            Vishalgad                     Khelna     Kolhapur   
 343      344           Worli Fort  Information Not Available  Mumbai City   
 344      345  Yashawantgad (Nate)  Information Not Available    Ratnagiri   
 345      346          Yashwantgad                  Redi Fort   Sindhudurg   
 
         taluka  latitude  longitude base_village             

In [26]:
clustered_df = cluster_engine.get_clustered_data()

In [27]:
clustered_df.iloc[0]

fort_id                                                                 1
name                                                             Aad Fort
alternate_names                                 Information Not Available
district                                                        Ratnagiri
taluka                                                             Dapoli
latitude                                                          17.7125
longitude                                                         73.2391
base_village                                                          Aad
type                                                Giri Durg (Hill Fort)
elevation_m                                                           250
current_condition                                                In Ruins
era                                                        Maratha Empire
built_by                                                          Unknown
year_of_construction                  